# Практическая работа 1. Источники данных. Извлечение данных



ФИО: Попова Екатерина Денисовна

Группа: АДЭУ-201

Используем данные из открытого источника для `CSV`, `Parquet` и `API`, а также вручную подготовим данные для баз данных и HTML  из открытых данных (доступно по адресу https://data.cityofnewyork.us).

In [ ]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
from unicodedata import normalize

1.1 Файлы данных `CSV` и `Excel`

Файлы данных могут быть получены откуда угодно: от файлов, хранящихся локально на вашем устройстве, до файловых систем облачного хранилища.

Даже при работе в основном с базами данных или внешними `API` использование физических файлов — отличный способ легко использовать данные с метками времени, что может пригодиться при любых временных проблемах с подключением.

In [ ]:
# Загрузите данные с портала открытых данных Правительства Москвы —
# данные о Wi-Fi в кинотеатрах, введя в браузер следующий URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false.
# Будет загружен CSV-файл *.csv. Сохраните этот файл.
# прочитать данные CSV, используя команду Pandas pd.read_csv() в качестве df DataFrame, затем выберите первые пять строк DataFrame с помощью df.head().

**Задание 1.1.** Зайти на сайт URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false

**Задание 1.2.** Загрузить данные о `Wi-Fi в кинотеатрах` в Colab.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving data.csv to data.csv


**Задание 1.3.** Считайте данные из файла CSV. Использовать метод `pandas read_csv` для простоты и скорости.

In [ ]:
df_csv = pd.read_csv("data.csv", delimiter=';')
df_csv.head()

,global_id,ID,CinemaName,AdmArea,District,Address,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Latitude_WGS84,Longitude_WGS84,geoData,geodata_center,Unnamed: 16
0,global_id,Код,Наименование кинотеатра,Административный округ,Район,Адрес,Количество точек доступа,Имя Wi-Fi сети,"Зона покрытия, в метрах",Признак функционирования,Условия доступа,Пароль,Широта в WGS-84,Долгота в WGS-84,geoData,geodata_center,NaN
1,879791944,1,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Останкинский район,"Российская Федерация, город Москва, внутригоро...",7,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.818675,37.636778,"{coordinates=[37.636778, 55.818675], type=Point}",NaN,NaN
2,879791951,3,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Печатники,"город Москва, улица Кухмистерова, дом 4",8,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.685873,37.718534,"{coordinates=[37.718534, 55.685873], type=Point}",NaN,NaN
3,879791953,4,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.710864,37.731299,"{coordinates=[37.731299, 55.710864], type=Point}",NaN,NaN
4,879791961,7,Государственное бюджетное учреждение культуры ...,Северный административный округ,Тимирязевский район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.814406,37.571210,"{coordinates=[37.57121, 55.814406], type=Point}",NaN,NaN


**Задание 1.4.** Так как первая строка включает определение столбцов, удалить строку с `index=0`.

In [ ]:
data = df_csv.drop(df_csv.index[0])

In [ ]:
data

,global_id,ID,CinemaName,AdmArea,District,Address,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Latitude_WGS84,Longitude_WGS84,geoData,geodata_center,Unnamed: 16
1,879791944,1,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Останкинский район,"Российская Федерация, город Москва, внутригоро...",7,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.818675,37.636778,"{coordinates=[37.636778, 55.818675], type=Point}",NaN,NaN
2,879791951,3,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Печатники,"город Москва, улица Кухмистерова, дом 4",8,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.685873,37.718534,"{coordinates=[37.718534, 55.685873], type=Point}",NaN,NaN
3,879791953,4,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.710864,37.731299,"{coordinates=[37.731299, 55.710864], type=Point}",NaN,NaN
4,879791961,7,Государственное бюджетное учреждение культуры ...,Северный административный округ,Тимирязевский район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.814406,37.571210,"{coordinates=[37.57121, 55.814406], type=Point}",NaN,NaN
5,879791967,9,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,район Свиблово,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.851945,37.647549,"{coordinates=[37.647549, 55.851945], type=Point}",NaN,NaN
6,879795206,2,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Бабушкинский район,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.861974,37.676825,"{coordinates=[37.676825, 55.861974], type=Point}",NaN,NaN
7,879795207,5,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",4,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.764303,37.705508,"{coordinates=[37.705508, 55.764303], type=Point}",NaN,NaN
8,879795208,6,Государственное бюджетное учреждение культуры ...,Юго-Западный административный округ,Академический район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.682787,37.571585,"{coordinates=[37.571585, 55.682787], type=Point}",NaN,NaN
9,879795209,8,Государственное бюджетное учреждение культуры ...,Восточный административный округ,район Новогиреево,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.749841,37.802562,"{coordinates=[37.802562, 55.749841], type=Point}",NaN,NaN
10,879795211,10,Государственное бюджетное учреждение культуры ...,Юго-Западный административный округ,Обручевский район,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.665309,37.513395,"{coordinates=[37.513395, 55.665309], type=Point}",NaN,NaN


# **Задание 1.5.** Выполнить парсинг веб-страницы с помощью `JSON`

In [ ]:
# Импорт стандартных пакетов
import pandas as pd

# Для парсинга веб-страниц
import requests as reqs
import urllib.request
from bs4 import BeautifulSoup
import json
import os
import pprint

# Для выполнения операций с регулярными выражениями
import re

In [ ]:
api_url = 'https://catfact.ninja/fact'

In [ ]:
resp = reqs.request(method='GET', url=api_url)
print(resp.json())

{'fact': 'It may take as long as 2 weeks for a kitten to be able to hear well.  Their eyes usually open between 7 and 10 days, but sometimes it happens in as little as 2 days.', 'length': 165}


## <font color='blue'>Парсинг веб-страниц с помощью JSON</font>

### <font color="green"> Что такое JSON?</font>

* JSON (нотация объектов JavaScript) — популярный формат данных, используемый для представления структурированных данных.
* Это текстовый формат, который не зависит от языка и может использоваться в Python, Perl и других языках.
* Формат JSON используется для передачи данных между серверами и веб-приложениями.
* Он построен на двух структурах:

      - Коллекция пар имя/значение. Это реализуется как объект, запись, словарь, хеш-таблица, список ключей или ассоциативный массив.
      - Упорядоченный список значений. Это реализуется как массив, вектор, список или последовательность.
     
     
Основные функции `JSON`:

* `dump()`: запись закодированной строки в файл.
* `load()`: декодирование во время чтения файла JSON.
* `dumps()`: кодирование в объекты JSON.
* `loads()`: декодирует строку JSON.

### <font color="green"> Сериализация и десериализация</font>

> … процесс перевода структур данных или состояния объекта в формат, который можно сохранить… или передать… и позже восстановить (возможно, в другой компьютерной среде). (Википедия)

* **Сериализация** — это процесс преобразования объектов или структур данных в потоки байтов или строки.
* Эти потоки байтов затем можно легко сохранить или передать.
* Это позволяет разработчикам сохранять, например, данные конфигурации или прогресс пользователя, а затем сохранять их (на диске или в базе данных) или отправлять в другое место.
* Обратный процесс сериализации известен как **десериализация**.

### Зачем нужна сериализация?

Нам нужна сериализация по следующим причинам:

- **Связь**: Сериализация включает в себя процедуру сериализации и передачи объекта. Это позволяет нескольким компьютерным системам одновременно проектировать, совместно использовать и выполнять объекты.
- **Кэширование**: время, затрачиваемое на создание объекта, больше времени, необходимого для его десериализации. Сериализация минимизирует затраты времени за счет кэширования гигантских объектов.
- **Глубокое копирование**: процесс клонирования упрощается благодаря сериализации. Точная копия объекта получается путем сериализации объекта в массив байтов и последующей его десериализации.
- **Переносимость**. Основное преимущество сериализации заключается в том, что она работает в разных архитектурах и операционных системах.
- **Постоянство**: состояние любого объекта можно сохранить напрямую, применив к нему сериализацию, и сохранить в базе данных, чтобы его можно было получить позже.

**Сериализация**

Мы используем функцию dump(), которая принимает два аргумента:
* Объект данных, подлежащий сериализации.
* Объект файла, в который он будет записан (байтовый формат).

In [ ]:
json_page = resp.json()
file_name = "fact.json"
with open(file_name, "w") as fid:
     json.dump(json_page, fid)

In [ ]:
!cat fact.json

{"fact": "It may take as long as 2 weeks for a kitten to be able to hear well.  Their eyes usually open between 7 and 10 days, but sometimes it happens in as little as 2 days.", "length": 165}

**Десериализация JSON**

* Десериализация противоположна сериализации, т. е. преобразованию объекта JSON в соответствующие объекты Python.
* Мы используем функцию load(), которая обычно используется для загрузки из строки, в противном случае корневой объект находится в списке или диктовке.

In [ ]:
with open(file_name, "r") as fid:
     json_obj = json.load(fid)

print(json_obj)

{'fact': 'It may take as long as 2 weeks for a kitten to be able to hear well.  Their eyes usually open between 7 and 10 days, but sometimes it happens in as little as 2 days.', 'length': 165}


In [ ]:
for key in json_obj:
    print("{:>12}: {}".format(key, json_obj[key]))

        fact: It may take as long as 2 weeks for a kitten to be able to hear well.  Their eyes usually open between 7 and 10 days, but sometimes it happens in as little as 2 days.
      length: 165


Обрабатываем данные с помощью `JSON`:

In [ ]:
response = reqs.get(api_url)

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.6.** Выполнить парсинг веб-страницы с помощью `JSON`. Получите URL-адрес:

In [ ]:
print(f"URL: \n\t {response.url}")

URL: 
	 https://catfact.ninja/fact


**Задание 1.7.** Выполнить парсинг веб-страницы с помощью `JSON`. Определите тип контента:

In [ ]:
print(f"Content type: \n\t {response.headers['content-type']}")

Content type: 
	 application/json


**Задание 1.8.** Выполнить парсинг веб-страницы с помощью `JSON`. Обработайте данные с помощью JSON:

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.9.**  Обработайте данные с помощью `JSON`. Посмотреть на ключи:

In [ ]:
for x in json_page:
    print(x)

fact
length


Распечатать ключи и значения:

In [ ]:
for x in json_page:
    print(f"{x} --> {json_page[x]} \n")

fact --> A cat's brain is more similar to a man's brain than that of a dog. 

length --> 66 



In [ ]:
pprint.pprint(json_page)

{'fact': "A cat's brain is more similar to a man's brain than that of a dog.",
 'length': 66}


# **Задание 1.10.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`

 <font color='blue'>Парсинг веб-страниц с помощью Beautiful Soup</font>

- Веб-скрапинг позволяет загрузить HTML-код веб-сайта и извлечь необходимые вам данные.
- Beautiful Soup — это библиотека Python для очистки данных с веб-сайтов.
- Beautiful Soup создает дерево разбора из проанализированных документов HTML и XML.

In [ ]:
from bs4 import BeautifulSoup as bso

In [ ]:
# Отправление запроса на получение данных и присваивание ответ переменной.
source = reqs.get("https://ru.ruwiki.ru/wiki/%D0%A0%D0%B5%D0%B9%D1%82%D0%B8%D0%BD%D0%B3_%D1%81%D1%82%D1%80%D0%B0%D0%BD_%D0%BF%D0%BE_%D1%87%D0%B8%D1%81%D0%BB%D1%83_%D0%BA%D0%BE%D0%BC%D0%BF%D1%8C%D1%8E%D1%82%D0%B5%D1%80%D0%BE%D0%B2_%D0%BD%D0%B0_1000_%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_(2004)")

In [ ]:
print(f"Content type: \n\t {source.headers['content-type']}")

Content type: 
	 text/html; charset=UTF-8


In [ ]:
print(source)

<Response [200]>


**Задание 1.10.1.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Создать beautiful soup объект.

In [ ]:
mysoup = bso(source.text, 'html.parser')

**Задание 1.10.2.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Распечатайте HTML-содержимое страницы, используя метод prettify

In [ ]:
print(mysoup.prettify())

<!DOCTYPE html>
<html class="client-nojs yenisey-feature-language-in-header-disabled yenisey-feature-language-in-main-page-header-disabled yenisey-feature-language-in-sidebar-disabled yenisey-feature-language-alert-in-sidebar-disabled yenisey-feature-sticky-header-disabled yenisey-feature-page-tools-disabled yenisey-feature-page-tools-pinned-disabled yenisey-feature-toc-pinned-enabled yenisey-feature-main-menu-pinned-disabled yenisey-feature-limited-width-enabled yenisey-feature-limited-width-content-enabled yenisey-feature-yenisey-is-header-top-message-enabled-disabled" dir="ltr" lang="ru">
 <head>
  <meta charset="utf-8"/>
  <title>
   Рейтинг стран по числу компьютеров на 1000 жителей (2004) — Рувики
  </title>
  <script>
   document.documentElement.className="client-js yenisey-feature-language-in-header-disabled yenisey-feature-language-in-main-page-header-disabled yenisey-feature-language-in-sidebar-disabled yenisey-feature-language-alert-in-sidebar-disabled yenisey-feature-sticky

**Задание 1.10.3.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Получить заголовок страницы

In [ ]:
print(mysoup.title)

<title>Рейтинг стран по числу компьютеров на 1000 жителей (2004) — Рувики</title>


# Получение данных из таблиц РСУБД

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving movies.sqlite to movies.sqlite


**Задание 1.11.** Считать результаты запроса sqlite в DataFrame pandas

In [ ]:
# Считываем результаты запроса sqlite в DataFrame pandas
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


# Получение данных `Parquet`

Посетите URL https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving yellow_tripdata_2023-03.parquet to yellow_tripdata_2023-03.parquet


**Задание 1.12.** Считайте данные из файла Parquet. Использовать метод pandas read_parquet.

In [ ]:
df_parquet = pd.read_parquet("yellow_tripdata_2023-03.parquet")
df_parquet.head()

# Индивидуальное задание

- Загрузить задание в среду colab.research

- Выполнить задания `1-11`, результаты сохранить в файле `ПР_1_ФИО.ipynb`.

- Выполнить парсинг веб-страниц с помощью Beautiful Soup и Pandas открытых источников данных:

1. Сайт Мосбиржи — основной сайт инвестора. Очень много полезной информации о ценных бумагах, валютах — https://www.moex.com/

2. Список биржевых инвестиционных фондов (ETF) на Московской бирже — https://www.moex.com/msn/etf

3. Сайт Санкт-Петербургской биржи — https://spbexchange.ru/

4. Список акций иностранных компаний, которые можно купить на Санкт-Петербургской бирже — http://stocks.spbexchange.ru/

5. ETF от FinEx — https://finex-etf.ru/products/

6. Котировки всех в мире акций, дивидендный календарь по акциям, курсы всех валют — https://ru.investing.com/

7. Котировки и курсы акций в режиме реального времени с фильтром по тем странам, которые вас интересуют, — https://ru.investing.com/equities/

8. Информация о дивидендах по российским акциям — https://www.dohod.ru/ik/analytics/dividend/

9. Дивиденды российских и американских компаний — https://investmint.ru/

10. Сервис с информацией по российским облигациям — https://bonds.finam.ru/

11. Анализ российских облигаций — https://www.dohod.ru/analytic/bonds

12. Котировки корпоративных облигаций — https://smart-lab.ru/q/bonds/

13. Иностранные акции, инвестиционные идеи — https://finviz.com/

14. Сервис для ведения портфеля ценных бумаг. Можно сделать публичный доступ и показать портфель своим друзьям, например — https://intelinvest.ru/

15. Журнал «Тинькофф». Информация о финансах, инвестициях, бесплатное обучение — https://journal.tinkoff.ru/pro/invest/

16. Обучающие курсы от Мосбиржи. Там же полезные сервисы в помощь инвесторам — https://red-circule.com/

17. Новости компаний и новая информация по акциям — https://smart-lab.ru/news/

18. Ожидаемые события фондового рынка — https://bcs-express.ru/ozhidaemye-sobytiya
Подробнее на сайте Banki.ru https://www.banki.ru/news/lenta/?id=10936184

19. Реестр сведений о банкротстве — https://bankrot.fedresurs.ru/

20. Реестр доверенностей — https://www.reestr-dover.ru/

21. Реестр уведомлений о залоге движимого имущества — https://www.reestr-zalogov.ru/state/index

22. Узнать свою кредитную историю — https://www.nbki.ru/serviceszaem/svedeniya/

23. Узнать задолженность на сайте ФССП. Не только свою — http://fssp.gov.ru/iss/ip/

24. О пособиях для безработных в 2020 году — https://www.gosuslugi.ru/help/news/2020_05_08_work_pay

25. Проект Банка России по финансовой грамотности — https://fincult.info/
Подробнее на сайте Banki.ru https://www.banki.ru/news/lenta/?id=1093618

- Выполнить парсинг коллекции данных на Портале открытых данных Правительства Москвы. Выявить показатели в наборе данных(изучить три датасета. Предоставить BI-табло результатов):

1. Торговля.

2. Дороги и транспорт.

3. ЖКХ.

4. Образование.

5. Трудоустройство

6. Здравоохранение.

7. Социальная среда.

8. Справочная информация.

9. Земля и имущество.

10. Культура.

11. Досух и отдых.

12. Предпринимательство.

13. Торговля-2.

14. Дороги и транспорт-2.

15. ЖКХ-2.

16. Образование-2.

17. Трудоустройство-2.

18. Здравоохранение-2.

19. Социальная среда-2.

20. Справочная информация-2.

21. Земля и имущество-2.

22. Культура-2.

23. Досух и отдых-2.

24. Предпринимательство-2.

25. Торговля-3.

